In [7]:
import pandas as pd
import numpy as np
import pulp

df=pd.read_csv("../data/dairy_cows.csv")
df.head()

,ID,Species,Animal_Class,Welfare_Hazards_Animal,Welfare_Hazards_Consequences,Welfare_Hazards_Impact,Ease_of_Hazard_Mitigation,Welfare_Indicator,Indicator_Ease,Indicator_Resources
0,1,Dairy cows,Tie stalls,Pasture access,Gastro-enteric disorders,High,Moderate,Abdominal discomfort,Moderate,Low
1,1,Dairy cows,Cubicles,Pasture access,Gastro-enteric disorders,High,Moderate,Abdominal discomfort,Moderate,Low
2,2,Dairy cows,Tie stalls,Continuous housing for long periods,General disruption of behaviour,Low,Difficult,Agonistic behaviour,Moderate,Low
3,2,Dairy cows,Cubicles,Continuous housing for long periods,General disruption of behaviour,Low,Difficult,Agonistic behaviour,Moderate,Low
4,3,Dairy cows,Tie stalls,Insufficient space,Restriction of movement,High,Moderate,Agonistic interactions,Moderate,Medium


## objective function

We aim to **maximize** a weighted sum that incorporates your criteria:

### Z=sum((alpha * impact) - (beta * ease) - (gamma * mitigation) - (deta*resources))


## Contraints

Select Exactly 4 Indicators

In [18]:
ease_mapping = {'Easy': 1, 'Moderate': 2, 'Difficult': 3}
resources_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
impact_mapping = {'Low': 1, 'High': 2}

# Apply mappings
df['Indicator_Ease_Num'] = df['Indicator_Ease'].map(ease_mapping)
df['Ease_of_Hazard_Mitigation_Num'] = df['Ease_of_Hazard_Mitigation'].map(ease_mapping)
df['Indicator_Resources_Num'] = df['Indicator_Resources'].map(resources_mapping)
df['Welfare_Hazards_Impact_Num'] = df['Welfare_Hazards_Impact'].map(impact_mapping)

In [19]:
indicators_list = df['Welfare_Indicator'].unique().tolist()

# List of unique Hazards (combination of Animal and Consequences)
df['Hazard'] = df['Welfare_Hazards_Animal'] + " | " + df['Welfare_Hazards_Consequences']
hazards_list = df['Hazard'].unique().tolist()

## Create Hazard-Indicator Association Matrix (alpha_h_i)
We'll create a dictionary to represent a_hi, indicating whether a hazard is addressed by an indicator.

In [20]:
a_hi = {h: {i: 0 for i in indicators_list} for h in hazards_list}

# Populate the dictionary
for idx, row in df.iterrows():
    h = row['Hazard']
    i = row['Welfare_Indicator']
    a_hi[h][i] = 1  # Indicator i addresses hazard h

In [22]:
indicator_attributes = df.groupby('Welfare_Indicator').agg({
    'Indicator_Ease_Num': 'mean',
    'Ease_of_Hazard_Mitigation_Num': 'mean',
    'Indicator_Resources_Num': 'mean'
}).to_dict('index')

In [23]:
hazard_impact = df.groupby('Hazard')['Welfare_Hazards_Impact_Num'].mean().to_dict()


In [24]:
prob = pulp.LpProblem("Welfare_Indicator_Selection", pulp.LpMaximize)

In [25]:
x_vars = pulp.LpVariable.dicts(
    "Select_Indicator", indicators_list, cat='Binary'
)

In [27]:
y_vars = pulp.LpVariable.dicts(
    "Cover_Hazard", hazards_list, cat='Binary'
)

In [28]:
alpha = 1.0  # Weight for hazard impact
beta = 0.5   # Weight for Indicator_Ease
gamma = 0.5  # Weight for Ease_of_Hazard_Mitigation
delta = 0.5  # Weight for Indicator_Resources


In [29]:
# Objective function components
hazard_impact_term = pulp.lpSum([hazard_impact[h] * y_vars[h] for h in hazards_list])
indicator_ease_term = pulp.lpSum([indicator_attributes[i]['Indicator_Ease_Num'] * x_vars[i] for i in indicators_list])
mitigation_ease_term = pulp.lpSum([indicator_attributes[i]['Ease_of_Hazard_Mitigation_Num'] * x_vars[i] for i in indicators_list])
resources_term = pulp.lpSum([indicator_attributes[i]['Indicator_Resources_Num'] * x_vars[i] for i in indicators_list])

# Define the objective function
prob += alpha * hazard_impact_term - beta * indicator_ease_term - gamma * mitigation_ease_term - delta * resources_term


In [30]:
prob += pulp.lpSum([x_vars[i] for i in indicators_list]) == 4, "Select_4_Indicators"


In [31]:
for h in hazards_list:
    prob += y_vars[h] <= pulp.lpSum([a_hi[h][i] * x_vars[i] for i in indicators_list]), f"Hazard_Coverage_{h}"


In [32]:
for h in hazards_list:
    prob += y_vars[h] >= (1 / len(hazards_list)) * pulp.lpSum([a_hi[h][i] * x_vars[i] for i in indicators_list]), f"Hazard_Coverage_Enforcement_{h}"


In [33]:
prob.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mohamedmentis/Dropbox/Mac (2)/Documents/Mentis/Development/Python/Wellfare_indicators/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0l/0_k60hjn72g4s56_ff84bnvw0000gn/T/a422b7d9e5884e379e39016fa43aab4e-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/0l/0_k60hjn72g4s56_ff84bnvw0000gn/T/a422b7d9e5884e379e39016fa43aab4e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 240 COLUMNS
At line 1420 RHS
At line 1656 BOUNDS
At line 1828 ENDATA
Problem MODEL has 235 rows, 171 columns and 666 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 69.8607 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 1011 substitutions
Cgl0004I processed model has 42 rows, 60 columns (60 integer (52 of which binary)) and 204 ele

1

onds):       0.01   (Wallclock seconds):       0.02



In [34]:
print("Status:", pulp.LpStatus[prob.status])

Status: Optimal


In [35]:
covered_hazards = [h for h in hazards_list if y_vars[h].varValue == 1]
print("\nCovered Hazards:")
for h in covered_hazards:
    print(f"- {h}")

# Total hazard impact covered
total_impact_covered = sum([hazard_impact[h] for h in covered_hazards])
print(f"\nTotal Hazard Impact Covered: {total_impact_covered}")



Covered Hazards:
- Continuous housing for long periods | General disruption of behaviour
- Early separation of cow and calf | Separation stress
- Early separation of cow and calf (after 24h) | Separation stress
- Inadequate bedding (cubicle/stall floor) | Environmental stress
- Inadequate design of waiting area (size, flooring, crowding gates) | Environmental stress
- Inadequate maintenance of milking equipment | Environmental stress
- Inadequate milking parlour design | Environmental stress
- Poor cubicle design | Environmental stress
- Poor stall design / stall too small | Environmental stress
- Use of cow trainers | Environmental stress
- Poor calving conditions (unable to separate from other animals) | Group (social) stress
- Lack of handling/restraining facilities | Handling stress
- Continuous housing for long periods | Locomotor disorders (including lameness)
- Inadequate flooring in passageways, feeding and milking areas | Locomotor disorders (including lameness)
- Mixing anim

In [36]:
print(f"\nObjective Function Value: {pulp.value(prob.objective)}")


Objective Function Value: 69.8607142857143


In [38]:
# Get the selected indicators
selected_indicators = [i for i in indicators_list if x_vars[i].varValue == 1]
print("Selected Welfare Indicators:")
for ind in selected_indicators:
    print(f"- {ind}")


print("\nSelected Indicators and Their Attributes:")
for ind in selected_indicators:
    attrs = indicator_attributes[ind]
    print(f"- {ind}:")
    print(f"  Indicator_Ease_Num: {attrs['Indicator_Ease_Num']}")
    print(f"  Ease_of_Hazard_Mitigation_Num: {attrs['Ease_of_Hazard_Mitigation_Num']}")
    print(f"  Indicator_Resources_Num: {attrs['Indicator_Resources_Num']}")

Selected Welfare Indicators:
- Hock alterations
- Injuries
- Physiological stress indicators
- Time budgets (disruption of behaviour)

Selected Indicators and Their Attributes:
- Hock alterations:
  Indicator_Ease_Num: 2.0
  Ease_of_Hazard_Mitigation_Num: 2.4285714285714284
  Indicator_Resources_Num: 1.0
- Injuries:
  Indicator_Ease_Num: 1.0
  Ease_of_Hazard_Mitigation_Num: 2.5
  Indicator_Resources_Num: 1.0
- Physiological stress indicators:
  Indicator_Ease_Num: 3.0
  Ease_of_Hazard_Mitigation_Num: 2.15
  Indicator_Resources_Num: 3.0
- Time budgets (disruption of behaviour):
  Indicator_Ease_Num: 2.0
  Ease_of_Hazard_Mitigation_Num: 2.2
  Indicator_Resources_Num: 2.0


In [39]:
# Hazards covered
covered_hazards = [h for h in hazards_list if y_vars[h].varValue == 1]
print("\nCovered Hazards:")
for h in covered_hazards:
    print(f"- {h}")

# Total hazard impact covered
total_impact_covered = sum([hazard_impact[h] for h in covered_hazards])
print(f"\nTotal Hazard Impact Covered: {total_impact_covered}")



Covered Hazards:
- Continuous housing for long periods | General disruption of behaviour
- Early separation of cow and calf | Separation stress
- Early separation of cow and calf (after 24h) | Separation stress
- Inadequate bedding (cubicle/stall floor) | Environmental stress
- Inadequate design of waiting area (size, flooring, crowding gates) | Environmental stress
- Inadequate maintenance of milking equipment | Environmental stress
- Inadequate milking parlour design | Environmental stress
- Poor cubicle design | Environmental stress
- Poor stall design / stall too small | Environmental stress
- Use of cow trainers | Environmental stress
- Poor calving conditions (unable to separate from other animals) | Group (social) stress
- Lack of handling/restraining facilities | Handling stress
- Continuous housing for long periods | Locomotor disorders (including lameness)
- Inadequate flooring in passageways, feeding and milking areas | Locomotor disorders (including lameness)
- Mixing anim

# Run2

In [41]:
df=pd.read_csv("../data/dairy_cows.csv")
df.head()

,ID,Species,Animal_Class,Welfare_Hazards_Animal,Welfare_Hazards_Consequences,Welfare_Hazards_Impact,Ease_of_Hazard_Mitigation,Welfare_Indicator,Indicator_Ease,Indicator_Resources
0,1,Dairy cows,Tie stalls,Pasture access,Gastro-enteric disorders,High,Moderate,Abdominal discomfort,Moderate,Low
1,1,Dairy cows,Cubicles,Pasture access,Gastro-enteric disorders,High,Moderate,Abdominal discomfort,Moderate,Low
2,2,Dairy cows,Tie stalls,Continuous housing for long periods,General disruption of behaviour,Low,Difficult,Agonistic behaviour,Moderate,Low
3,2,Dairy cows,Cubicles,Continuous housing for long periods,General disruption of behaviour,Low,Difficult,Agonistic behaviour,Moderate,Low
4,3,Dairy cows,Tie stalls,Insufficient space,Restriction of movement,High,Moderate,Agonistic interactions,Moderate,Medium


In [51]:
import pandas as pd
import numpy as np
import pulp

# Encode ordinal variables
ease_mapping = {'Easy': 1, 'Moderate': 2, 'Difficult': 3}
resources_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
impact_mapping = {'Low': 1, 'High': 2}

df['Indicator_Ease_Num'] = df['Indicator_Ease'].map(ease_mapping)
df['Ease_of_Hazard_Mitigation_Num'] = df['Ease_of_Hazard_Mitigation'].map(ease_mapping)
df['Indicator_Resources_Num'] = df['Indicator_Resources'].map(resources_mapping)
df['Welfare_Hazards_Impact_Num'] = df['Welfare_Hazards_Impact'].map(impact_mapping)

# Identify unique Welfare Indicators, Hazards, and Consequences
indicators_list = df['Welfare_Indicator'].unique().tolist()
hazards_animal_list = df['Welfare_Hazards_Animal'].unique().tolist()
hazards_consequence_list = df['Welfare_Hazards_Consequences'].unique().tolist()

# Create hazard-indicator association matrices
# For animal hazards
a_hi_animal = {h: {i: 0 for i in indicators_list} for h in hazards_animal_list}
for idx, row in df.iterrows():
    h = row['Welfare_Hazards_Animal']
    i = row['Welfare_Indicator']
    a_hi_animal[h][i] = 1

# For consequences
a_ci_consequence = {c: {i: 0 for i in indicators_list} for c in hazards_consequence_list}
for idx, row in df.iterrows():
    c = row['Welfare_Hazards_Consequences']
    i = row['Welfare_Indicator']
    a_ci_consequence[c][i] = 1

# Prepare parameter dictionaries
# Aggregate indicator attributes
indicator_attributes = df.groupby('Welfare_Indicator').agg({
    'Indicator_Ease_Num': 'mean',
    'Ease_of_Hazard_Mitigation_Num': 'mean',
    'Indicator_Resources_Num': 'mean'
}).to_dict('index')

# Hazard impact for animal hazards
hazard_animal_impact = df.groupby('Welfare_Hazards_Animal')['Welfare_Hazards_Impact_Num'].mean().to_dict()

# Hazard impact for consequences
hazard_consequence_impact = df.groupby('Welfare_Hazards_Consequences')['Welfare_Hazards_Impact_Num'].mean().to_dict()

# Initialize the problem
prob = pulp.LpProblem("Welfare_Indicator_Selection", pulp.LpMaximize)

# Decision variables
x_vars = pulp.LpVariable.dicts("Select_Indicator", indicators_list, cat='Binary')
y_vars_animal = pulp.LpVariable.dicts("Cover_Hazard_Animal", hazards_animal_list, cat='Binary')
y_vars_consequence = pulp.LpVariable.dicts("Cover_Hazard_Consequence", hazards_consequence_list, cat='Binary')

# Set weights
alpha_animal = 1.0  # Weight for animal hazard impact
alpha_consequence = 1.0  # Weight for consequence impact
beta = 1.0   # Weight for Indicator_Ease
gamma = 1.0  # Weight for Ease_of_Hazard_Mitigation
delta = 1.0  # Weight for Indicator_Resources

# Objective function components
hazard_animal_impact_term = pulp.lpSum([hazard_animal_impact[h] * y_vars_animal[h] for h in hazards_animal_list])
hazard_consequence_impact_term = pulp.lpSum([hazard_consequence_impact[c] * y_vars_consequence[c] for c in hazards_consequence_list])
indicator_ease_term = pulp.lpSum([indicator_attributes[i]['Indicator_Ease_Num'] * x_vars[i] for i in indicators_list])
mitigation_ease_term = pulp.lpSum([indicator_attributes[i]['Ease_of_Hazard_Mitigation_Num'] * x_vars[i] for i in indicators_list])
resources_term = pulp.lpSum([indicator_attributes[i]['Indicator_Resources_Num'] * x_vars[i] for i in indicators_list])

# Define the objective function
prob += (
    alpha_animal * hazard_animal_impact_term +
    alpha_consequence * hazard_consequence_impact_term -
    beta * indicator_ease_term -
    gamma * mitigation_ease_term -
    delta * resources_term
)

# Constraints
# 1. Select exactly 4 indicators
prob += pulp.lpSum([x_vars[i] for i in indicators_list]) == 4, "Select_4_Indicators"

# 2. Hazard coverage constraints for animal hazards
for h in hazards_animal_list:
    prob += y_vars_animal[h] <= pulp.lpSum([a_hi_animal[h][i] * x_vars[i] for i in indicators_list]), f"Hazard_Animal_Coverage_{h}"
    prob += y_vars_animal[h] >= (1 / len(hazards_animal_list)) * pulp.lpSum([a_hi_animal[h][i] * x_vars[i] for i in indicators_list]), f"Hazard_Animal_Coverage_Enforcement_{h}"

# 3. Hazard coverage constraints for consequences
for c in hazards_consequence_list:
    prob += y_vars_consequence[c] <= pulp.lpSum([a_ci_consequence[c][i] * x_vars[i] for i in indicators_list]), f"Hazard_Consequence_Coverage_{c}"
    prob += y_vars_consequence[c] >= (1 / len(hazards_consequence_list)) * pulp.lpSum([a_ci_consequence[c][i] * x_vars[i] for i in indicators_list]), f"Hazard_Consequence_Coverage_Enforcement_{c}"

# Solve the problem
prob.solve()

# Check status
print("Status:", pulp.LpStatus[prob.status])

# Retrieve selected indicators
selected_indicators = [i for i in indicators_list if x_vars[i].varValue == 1]
print("\nSelected Welfare Indicators:")
for ind in selected_indicators:
    print(f"- {ind}")

# Analyze hazard coverage
# Animal hazards
covered_hazards_animal = [h for h in hazards_animal_list if y_vars_animal[h].varValue == 1]
print("\nCovered Animal Hazards:")
for h in covered_hazards_animal:
    print(f"- {h}")

total_hazards_animal_covered = len(covered_hazards_animal)
print(f"\nTotal Number of Animal Hazards Covered: {total_hazards_animal_covered}")

# Consequences
covered_hazards_consequence = [c for c in hazards_consequence_list if y_vars_consequence[c].varValue == 1]
print("\nCovered Consequences:")
for c in covered_hazards_consequence:
    print(f"- {c}")

total_hazards_consequence_covered = len(covered_hazards_consequence)
print(f"\nTotal Number of Consequences Covered: {total_hazards_consequence_covered}")

# Total hazard impact covered
total_impact_animal = sum([hazard_animal_impact[h] for h in covered_hazards_animal])
total_impact_consequence = sum([hazard_consequence_impact[c] for c in covered_hazards_consequence])
total_impact_covered = total_impact_animal + total_impact_consequence

print(f"\nTotal Hazard Impact from Animal Hazards Covered: {total_impact_animal}")
print(f"Total Hazard Impact from Consequences Covered: {total_impact_consequence}")
print(f"Total Hazard Impact Covered: {total_impact_covered}")

# Objective function value
print(f"\nObjective Function Value: {pulp.value(prob.objective)}")

# Display selected indicators and their attributes
print("\nSelected Indicators and Their Attributes:")
for ind in selected_indicators:
    attrs = indicator_attributes[ind]
    print(f"- {ind}:")
    print(f"  Indicator_Ease_Num: {attrs['Indicator_Ease_Num']}")
    print(f"  Ease_of_Hazard_Mitigation_Num: {attrs['Ease_of_Hazard_Mitigation_Num']}")
    print(f"  Indicator_Resources_Num: {attrs['Indicator_Resources_Num']}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mohamedmentis/Dropbox/Mac (2)/Documents/Mentis/Development/Python/Wellfare_indicators/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0l/0_k60hjn72g4s56_ff84bnvw0000gn/T/2422da0973464a87b873f75e3e3f0466-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/0l/0_k60hjn72g4s56_ff84bnvw0000gn/T/2422da0973464a87b873f75e3e3f0466-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 142 COLUMNS
At line 1199 RHS
At line 1337 BOUNDS
At line 1460 ENDATA
Problem MODEL has 137 rows, 122 columns and 690 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 52.0522 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 125 substitutions
Cgl0004I processed model has 80 rows, 78 columns (78 integer (69 of which binary)) and 461 elem

In [52]:
# Total number of unique animal hazards
total_hazards_animal = len(hazards_animal_list)

# Total number of unique consequences
total_hazards_consequence = len(hazards_consequence_list)

# Number of animal hazards covered
total_hazards_animal_covered = len(covered_hazards_animal)

# Number of consequences covered
total_hazards_consequence_covered = len(covered_hazards_consequence)


In [53]:
# Percentage coverage for animal hazards
percentage_coverage_animal = (total_hazards_animal_covered / total_hazards_animal) * 100

# Percentage coverage for consequences
percentage_coverage_consequence = (total_hazards_consequence_covered / total_hazards_consequence) * 100

In [55]:
# Total number of unique animal hazards
total_hazards_animal = len(hazards_animal_list)

# Percentage coverage for animal hazards
percentage_coverage_animal = (total_hazards_animal_covered / total_hazards_animal) * 100

print(f"\nPercentage of Animal Hazards Covered: {percentage_coverage_animal:.2f}%")

# Total number of unique consequences
total_hazards_consequence = len(hazards_consequence_list)

# Percentage coverage for consequences
percentage_coverage_consequence = (total_hazards_consequence_covered / total_hazards_consequence) * 100


percentage_coverage=((total_hazards_animal_covered+total_hazards_consequence_covered)/(total_hazards_animal+total_hazards_consequence))*100
print(f"Percentage of Consequences Covered: {percentage_coverage_consequence:.2f}%")

print(f"Percentage Covered: {percentage_coverage:.2f}%")


Percentage of Animal Hazards Covered: 80.43%
Percentage of Consequences Covered: 54.55%
Percentage Covered: 72.06%
